# Naver blog scrap

## 라이브러리

In [1]:
import pandas as pd
import time, os, random
from datetime import datetime
from numpy import nan

In [2]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import chromedriver_autoinstaller as ca

## 게시글 주소 입력

In [3]:
# 게시글 주소
url = 'https://blog.naver.com/PostView.naver?blogId=icouncil103&logNo=222715190906&categoryNo=9&parentCategoryNo=9&from=thumbnailList'

## 크롤링 함수

In [4]:
def crawling(comment_areas, list):
    # 리스트 지정
    date_list = list[0]
    rare_number_list = list[1]
    check_reply_list = list[2]
    nickname_list = list[3]
    writer_url_list = list[4]
    like_num_list = list[5]
    link_num_list = list[6]
    reply_num_list = list[7]
    img_list = list[8]
    comment_list = list[9]

    for one in comment_areas:
        # 작성일 수집
        date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
        date_list.append(date)

        # 고유번호
        temp = one.attrs['class'][1].split('_')[-1]
        rare_number_list.append(temp)

        # 답글 구분
        try:
            one.find('span', 'u_cbox_secret_contents').text
            temp = '비밀댓글'
        except:
            try:
                temp = one.find('div', 'u_cbox_reply_area').find('li')
                temp = nan
            except:
                temp = '답글'
        check_reply_list.append(temp)

        # 닉네임
        try:
            temp = one.find('span', 'u_cbox_nick').text
        except:
            temp = '비밀댓글'
        nickname_list.append(temp)

        # 작성자 url
        try:
            temp = one.find('a', 'u_cbox_name').attrs['href']
        except:
            temp = '비밀댓글'
        writer_url_list.append(temp)

        # 공감수
        try:
            temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
        except:
            temp = 0
        like_num_list.append(temp)

        # 링크 수
        try:
            temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
            temp = len(temp)
        except:
            temp = 0
        link_num_list.append(temp)

        # 답글 수
        try:
            temp = one.find('div', 'u_cbox_reply_area').find_all('li')
            temp = len(temp)
        except:
            temp = 0
        reply_num_list.append(temp)

        # 첨부이미지url
        try:
            temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
        except:
            temp = nan
        img_list.append(temp)

        # 댓글 내용 수집
        try:
            temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
        except:
            temp = '비밀댓글'
        comment_list.append(temp)

    # 수집한 데이터 확인
    print('작성일 리스트 길이 : ', len(date_list))
    print('고유번호 리스트 길이 : ', len(rare_number_list))
    print('답글구분 리스트 길이 : ', len(check_reply_list))
    print('닉네임 리스트 길이 : ', len(nickname_list))
    print('작성자URL 리스트 길이 : ', len(writer_url_list))
    print('공감수 리스트 길이 : ', len(like_num_list))
    print('링크수 리스트 길이 : ', len(link_num_list))
    print('답글수 리스트 길이 : ', len(reply_num_list))
    print('첨부이미지URL 리스트 길이 : ', len(img_list))
    print('내용 리스트 길이 : ', len(comment_list))

    # 리스트에 담기
    crawled_list = [date_list, rare_number_list, check_reply_list, nickname_list, 
            writer_url_list, like_num_list, link_num_list, reply_num_list,
            img_list, comment_list]

    return crawled_list

## 현재 파일 위치 확인

In [5]:
# 현재 파일 위치 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/incheon'

## 수집한 파일 저장 폴더 생성

In [6]:
# 수집한 파일을 저장할 폴더 생성
crawled_folder_path = code_path + '/crawled_data/naver_blog'
os.makedirs(crawled_folder_path, exist_ok=True)

## 크롬 드라이버 버전 확인 및 설치

In [7]:
# 크롬 드라이버 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'100'

In [8]:
# 크롬 드라이버 설치 확인
try:
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')
except:
    ca.install(True)
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')

C:\Users\kbjung\AppData\Local\Temp/ipykernel_12788/3261653714.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')


## 페이지 접속

In [9]:
# 페이지 접속
driver.get(url)
driver.implicitly_wait(3)
driver.maximize_window()

In [10]:
# page 정보
page_info = url.split('=')[2].split('&')[0]
page_info

'222715190906'

## 페이지 스크래핑

### 댓글 페이지 개수

In [11]:
# 댓글 버튼 클릭
comment_button = driver.find_element_by_xpath(f'//*[@id="Comi{page_info}"]')
comment_button.click()
time.sleep( random.uniform(0.5, 1) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_12788/4256789063.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  comment_button = driver.find_element_by_xpath(f'//*[@id="Comi{page_info}"]')


In [12]:
soup = bs(driver.page_source, 'lxml')
num_page_buttons = int(soup.find('span', '_lastPageNo').text)
num_page_buttons

10

In [13]:
soup.title.text

'[이벤트] 인천시의회! 얼마나 알고 있나요? OX 퀴즈 : 네이버 블로그'

In [14]:
if num_page_buttons <= 10:
    prev_button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}"]/div/div[4]/div/a[2]')
    try:
        prev_button.click()
        time.sleep( random.uniform(0.5, 1) )
    except:
        pass
elif num_page_buttons > 10:
    prev_button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}"]/div/div[4]/div/a[2]')
    for i in range(num_page_buttons // 10 + 1):
        try:
            prev_button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}"]/div/div[4]/div/a[2]')
            prev_button.click()
            time.sleep( random.uniform(0.5, 1) )
        except:
            pass

C:\Users\kbjung\AppData\Local\Temp/ipykernel_12788/1503457795.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  prev_button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}"]/div/div[4]/div/a[2]')


In [15]:
list = [[], [], [], [], [], [], [], [], [], []]
len(list), list

(10, [[], [], [], [], [], [], [], [], [], []])

In [16]:
# 페이지 이동하며 정보 크롤링
# 1페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
num_current_page = int( comment_page_soup.find('strong', '_currentPageNo').text )
print( f"{num_current_page} 페이지 댓글 수 :", len(comment_areas) )
# 정보 크롤링
list = crawling(comment_areas, list)

for i in range(num_page_buttons - 1):
    try:
        # 페이지 이동
        button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}_ct"]/div[1]/div/div[2]/a[2]')
        button.click()
        driver.implicitly_wait(1)
        time.sleep( random.uniform(0.5, 1) )

        # 페이지 수프에 담기
        comment_page_soup = bs(driver.page_source, 'lxml')
        comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
        num_current_page = int( comment_page_soup.find('strong', '_currentPageNo').text )
        print( f"{num_current_page} 페이지 댓글 수 :", len(comment_areas) )
        # 정보 크롤링
        list = crawling(comment_areas, list)
    except:
        pass

1 페이지 댓글 수 : 50
작성일 리스트 길이 :  50
고유번호 리스트 길이 :  50
답글구분 리스트 길이 :  50
닉네임 리스트 길이 :  50
작성자URL 리스트 길이 :  50
공감수 리스트 길이 :  50
링크수 리스트 길이 :  50
답글수 리스트 길이 :  50
첨부이미지URL 리스트 길이 :  50
내용 리스트 길이 :  50


C:\Users\kbjung\AppData\Local\Temp/ipykernel_12788/2623418623.py:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = driver.find_element_by_xpath(f'//*[@id="naverComment_201_{page_info}_ct"]/div[1]/div/div[2]/a[2]')


2 페이지 댓글 수 : 50
작성일 리스트 길이 :  100
고유번호 리스트 길이 :  100
답글구분 리스트 길이 :  100
닉네임 리스트 길이 :  100
작성자URL 리스트 길이 :  100
공감수 리스트 길이 :  100
링크수 리스트 길이 :  100
답글수 리스트 길이 :  100
첨부이미지URL 리스트 길이 :  100
내용 리스트 길이 :  100
3 페이지 댓글 수 : 50
작성일 리스트 길이 :  150
고유번호 리스트 길이 :  150
답글구분 리스트 길이 :  150
닉네임 리스트 길이 :  150
작성자URL 리스트 길이 :  150
공감수 리스트 길이 :  150
링크수 리스트 길이 :  150
답글수 리스트 길이 :  150
첨부이미지URL 리스트 길이 :  150
내용 리스트 길이 :  150
4 페이지 댓글 수 : 50
작성일 리스트 길이 :  200
고유번호 리스트 길이 :  200
답글구분 리스트 길이 :  200
닉네임 리스트 길이 :  200
작성자URL 리스트 길이 :  200
공감수 리스트 길이 :  200
링크수 리스트 길이 :  200
답글수 리스트 길이 :  200
첨부이미지URL 리스트 길이 :  200
내용 리스트 길이 :  200
5 페이지 댓글 수 : 50
작성일 리스트 길이 :  250
고유번호 리스트 길이 :  250
답글구분 리스트 길이 :  250
닉네임 리스트 길이 :  250
작성자URL 리스트 길이 :  250
공감수 리스트 길이 :  250
링크수 리스트 길이 :  250
답글수 리스트 길이 :  250
첨부이미지URL 리스트 길이 :  250
내용 리스트 길이 :  250
6 페이지 댓글 수 : 50
작성일 리스트 길이 :  300
고유번호 리스트 길이 :  300
답글구분 리스트 길이 :  300
닉네임 리스트 길이 :  300
작성자URL 리스트 길이 :  300
공감수 리스트 길이 :  300
링크수 리스트 길이 :  300
답글수 리스트 길이 :  300
첨부이미지URL 리스트 길이

## 수집한 데이터 확인

In [17]:
print('작성일 리스트 길이 : ', len(list[0]))
print('고유번호 리스트 길이 : ', len(list[1]))
print('답글구분 리스트 길이 : ', len(list[2]))
print('닉네임 리스트 길이 : ', len(list[3]))
print('작성자URL 리스트 길이 : ', len(list[4]))
print('공감수 리스트 길이 : ', len(list[5]))
print('링크수 리스트 길이 : ', len(list[6]))
print('답글수 리스트 길이 : ', len(list[7]))
print('첨부이미지URL 리스트 길이 : ', len(list[8]))
print('내용 리스트 길이 : ', len(list[9]))

작성일 리스트 길이 :  498
고유번호 리스트 길이 :  498
답글구분 리스트 길이 :  498
닉네임 리스트 길이 :  498
작성자URL 리스트 길이 :  498
공감수 리스트 길이 :  498
링크수 리스트 길이 :  498
답글수 리스트 길이 :  498
첨부이미지URL 리스트 길이 :  498
내용 리스트 길이 :  498


## df 만들기

In [18]:
dict = {'작성일':list[0], '고유번호':list[1], 
        '답글구분':list[2], '닉네임':list[3], 
        '작성자URL':list[4], '공감수':list[5], 
        '링크수':list[6], '답글수':list[7], 
        '첨부이미지URL':list[8], '내용':list[9]}
df = pd.DataFrame(dict)

In [19]:
df.head()

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
0,2022-04-29 11:03:26,756465080060608740,NaN,치즈닭강정,https://blog.naver.com/junb85,0,1,0,NaN,1. X2. O3. O참여합니다! https://blog.naver.com/junb...
1,2022-04-29 11:06:31,756465277914317078,NaN,봄이s,https://blog.naver.com/love1004gpal,0,0,0,NaN,XOO
2,2022-04-29 11:07:18,756465328917053600,NaN,강냉콩,https://blog.naver.com/realpunica,0,1,1,https://g-cbox.pstatic.net/MjAyMjA0MjlfMjE0/MD...,1. X2. O3. O인천광역시의회 블로그 이웃이에요지방선거가 다가오고 있네요이번 ...
3,2022-04-29 11:09:10,756465450014998613,답글,강냉콩,https://blog.naver.com/realpunica,0,0,0,https://g-cbox.pstatic.net/MjAyMjA0MjlfNTkg/MD...,트위터 팔로우 완료
4,2022-04-29 11:09:36,756465476690772256,NaN,설향맘,https://blog.naver.com/hyss9744,0,1,3,https://g-cbox.pstatic.net/MjAyMjA0MjlfMjcx/MD...,정답 1) X2) O3) O @작가 수연 @여인네 함께해요 https://m.bl...


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       498 non-null    object
 1   고유번호      498 non-null    object
 2   답글구분      17 non-null     object
 3   닉네임       498 non-null    object
 4   작성자URL    498 non-null    object
 5   공감수       498 non-null    int64 
 6   링크수       498 non-null    int64 
 7   답글수       498 non-null    int64 
 8   첨부이미지URL  73 non-null     object
 9   내용        498 non-null    object
dtypes: int64(3), object(7)
memory usage: 39.0+ KB


### df 확인
* 고유번호 : undefined 제거
* 닉네임 : 비밀댓글 제거
* 답글 구분 : 비밀댓글 nan값으로 교체

#### 고유번호에서 undefined 제거

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       498 non-null    object
 1   고유번호      498 non-null    object
 2   답글구분      17 non-null     object
 3   닉네임       498 non-null    object
 4   작성자URL    498 non-null    object
 5   공감수       498 non-null    int64 
 6   링크수       498 non-null    int64 
 7   답글수       498 non-null    int64 
 8   첨부이미지URL  73 non-null     object
 9   내용        498 non-null    object
dtypes: int64(3), object(7)
memory usage: 39.0+ KB


In [22]:
df[df['고유번호'] == 'undefined']

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용


In [23]:
undefined_index = df[df['고유번호'] == 'undefined'].index
undefined_index

Int64Index([], dtype='int64')

In [24]:
df2 = df.drop(index=undefined_index).reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       498 non-null    object
 1   고유번호      498 non-null    object
 2   답글구분      17 non-null     object
 3   닉네임       498 non-null    object
 4   작성자URL    498 non-null    object
 5   공감수       498 non-null    int64 
 6   링크수       498 non-null    int64 
 7   답글수       498 non-null    int64 
 8   첨부이미지URL  73 non-null     object
 9   내용        498 non-null    object
dtypes: int64(3), object(7)
memory usage: 39.0+ KB


#### 닉네임에서 비밀댓글 제거

In [25]:
df2[df2['닉네임'] == '비밀댓글']

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
36,2022-04-29 12:03:52,756468973263913362,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
205,2022-04-29 17:03:04,756488248959696926,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
239,2022-04-29 22:56:21,756511009887224096,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
318,2022-04-30 16:28:38,756578802221449439,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
344,2022-04-30 22:37:37,756602574429225289,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
383,2022-05-01 21:37:56,756691499713823824,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글
384,2022-05-01 21:37:59,756691503622915325,비밀댓글,비밀댓글,비밀댓글,0,0,0,NaN,비밀댓글


In [26]:
secret_reply_index = df2[df2['닉네임'] == '비밀댓글'].index
secret_reply_index

Int64Index([36, 205, 239, 318, 344, 383, 384], dtype='int64')

In [27]:
df3 = df2.drop(index=secret_reply_index).reset_index(drop=True)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       491 non-null    object
 1   고유번호      491 non-null    object
 2   답글구분      10 non-null     object
 3   닉네임       491 non-null    object
 4   작성자URL    491 non-null    object
 5   공감수       491 non-null    int64 
 6   링크수       491 non-null    int64 
 7   답글수       491 non-null    int64 
 8   첨부이미지URL  73 non-null     object
 9   내용        491 non-null    object
dtypes: int64(3), object(7)
memory usage: 38.5+ KB


In [28]:
df3['답글구분'].unique()

array([nan, '답글'], dtype=object)

#### 답글구분에서 비밀댓글 nan 값으로 교체

In [29]:
df3[df3['답글구분'] == '비밀댓글']

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용


In [30]:
index3 = df3[df3['답글구분'] == '비밀댓글'].index
index3

Int64Index([], dtype='int64')

In [31]:
df4 = df3.replace( {'답글구분':'비밀댓글'}, nan )

In [32]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   작성일       491 non-null    object
 1   고유번호      491 non-null    object
 2   답글구분      10 non-null     object
 3   닉네임       491 non-null    object
 4   작성자URL    491 non-null    object
 5   공감수       491 non-null    int64 
 6   링크수       491 non-null    int64 
 7   답글수       491 non-null    int64 
 8   첨부이미지URL  73 non-null     object
 9   내용        491 non-null    object
dtypes: int64(3), object(7)
memory usage: 38.5+ KB


In [33]:
final_df = df4.copy()

## 파일로 출력

In [34]:
crawled_folder_path

'c:/data/Wantreez/Crawling/incheon/crawled_data/naver_blog'

In [35]:
today_date = datetime.today().strftime('%Y%m%d_%H%M%S')
today_date

'20220504_125535'

In [36]:
today_date = datetime.today().strftime('%Y%m%d_%H%M%S')
file_name = f'naver_blog_{today_date}.xlsx'
final_df.to_excel(crawled_folder_path + '/' + file_name, index=False, encoding='utf-8')
print(f'{file_name} 출력완료')

naver_blog_20220504_125535.xlsx 출력완료


# 코드 마지막